In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# Uma camada de convolução: demonstração
nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=(1,2))

Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 2))

In [4]:
# Classe para o modelo de convolução:
class ModeloCnn(nn.Module): # hereins from nn.Module
    def __init__(self):
        super().__init__()
        # Camada de entrada
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # Primeira Camada Oculta
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # Segunda Camada Oculta
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # camada convolução
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Transform linear 1
        self.linear1 = nn.Linear(in_features= 64 * 4 * 4, out_features=512)
        # Transform Linear 2
        self.linear2 = nn.Linear(in_features=512, out_features=10)
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
    
    def forward(self, x):
        # primeira parte - convolução
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        # segunda parte - Dense Layer
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return( x )

### Preparando os dados

In [5]:
# pipeline de transformações:
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

### ETL and split of Data

In [6]:
# Load trains and test data
train_data = datasets.CIFAR10("CIFAR10", train = True, download = True, transform = transformations )
test_data = datasets.CIFAR10("CIFAR10", train = False, download = True, transform = transformations )

2.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

4.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

15.5%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

23.8%IOPub m

95.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

97.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Extracting CIFAR10\cifar-10-python.tar.gz to CIFAR10
Files already downloaded and verified


In [42]:
len(train_data), len(test_data)

(50000, 10000)

In [43]:
validation_size = 0.2 
training_size = len(train_data)

In [44]:
# index for training and validation
indices = list(range(training_size))
np.random.shuffle(indices)
index_split = int(np.floor(training_size * validation_size))

In [48]:
# creating subset sampler for training and validation objects
validation_indices, training_indices = indices[:index_split], indices[index_split:]
training_sample, validation_sample = SubsetRandomSampler(training_indices), SubsetRandomSampler(validation_indices)

In [51]:
# Geras as amostras de dados para treino e validação e os devidos batches
train_loader = DataLoader(train_data, batch_size = 16, sampler = training_sample)
valid_loader = DataLoader(train_data, batch_size = 16, sampler = validation_sample)
test_loader = DataLoader(test_data, batch_size=16)

### Otimização do modelo

In [54]:
model = ModeloCnn()
model

ModeloCnn(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [55]:
# identificação do dispositivo:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cpu'

In [56]:
# se o device for gpu, faça:
# model = model.to(device)

In [57]:
# Função de custo:
criterion = nn.CrossEntropyLoss()

In [59]:
# optmizer backward
optmizer = optim.SGD(params= model.parameters(), lr = 0.01)

In [ ]:
# Accuracy Function
# o modelo retorna a probabilidade de cada classe (preds)
def accuracy(preds, y):
    _, pred = torch.argmax(preds, 1)
    correct = pred.eq(target.data.view_as(pred))
    acc = correct.sum() / len(correct)
    return acc

### Training